# Importing libraries and dataset

In [1]:
# IMPORT
import scipy.io as sio
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import torch.utils.data as dataloader
import matplotlib.pyplot as plt
import time
import os
import shutil
import copy
import random
from random import shuffle
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

In [2]:
dataset_cifar = torchvision.datasets.CIFAR10('/content/Cifar10_root', download=True)

100%|██████████| 170498071/170498071 [00:12<00:00, 13127460.87it/s]


Extracting /content/Cifar10_root/cifar-10-python.tar.gz to /content/Cifar10_root


In [3]:

print(len(dataset_cifar))

50000


In [4]:
torch.manual_seed(0)
np.random.seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [5]:

print(torch.__version__)
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

2.0.1+cu118
True
Tesla T4
cuda:0


# Data preparation

In [7]:
train_set, val_set = torch.utils.data.random_split(dataset_cifar, [40000, 10000])

after a random split in 70% and 30% for train and validation we will apply the data augmentation tecnique

In [8]:
train_images=[]
convert_tensor = transforms.ToTensor()

for images,targets in train_set:
  train_images.append(convert_tensor(images))
  im2 = np.fliplr(images).copy()
  train_images.append(convert_tensor(im2))

valid_images = []
for images,targets in val_set:
  valid_images.append(convert_tensor(images))
  im2 = np.fliplr(images).copy()
  valid_images.append(convert_tensor(im2))

after data augmentation we apply a normalization

In [9]:

norm = transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))
for elem in train_images:
  elem = norm(elem)

for elem in valid_images:
  elem = norm(elem)

extraction of classes

In [10]:
target=[]
for _,targets in train_set:
  target.append(targets)
  target.append(targets)

target2=[]
for _,targets in val_set:
  target2.append(targets)
  target2.append(targets)

creation of a Dataset

In [12]:



class MyDataset(Dataset):
    def __init__(self, data, targets):
        self.data = data
        self.targets = targets

    def __getitem__(self, index):
        x = self.data[index]
        y = self.targets[index]



        return x, y

    def __len__(self):
        return len(self.data)

In [13]:
train_data = MyDataset(train_images,target)
val_data = MyDataset(valid_images,target2)

Creation of a CNN from scratch

In [14]:

class CNNfdp(nn.Module):
  def __init__(self,num_classes):
    super(CNNfdp, self).__init__()
    self.conv_layer1 = nn.Conv2d(in_channels=3, out_channels=128, kernel_size=3, padding=2)
    self.conv_layer2 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=2)
    self.conv_layer3 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=2)

    self.maxpool_lyr = nn.MaxPool2d(kernel_size=2, stride=2)
    self.dropout_lyr = nn.Dropout(0.25)

    self.fc1 = nn.Linear(12800, 128)
    self.fc_final = nn.Linear(128, num_classes)


  def forward(self, x):
    x = self.maxpool_lyr(F.relu(self.conv_layer1(x)))
    x = self.maxpool_lyr(F.relu(self.conv_layer2(x)))
    x = self.maxpool_lyr(F.relu(self.conv_layer3(x)))
    x= x.view(-1, 12800)
    x = F.relu(self.fc1(x))
    x = self.dropout_lyr(x)
    x = self.fc_final(x)
    return x

creation of a Dataloader

In [15]:

Batch_size = 16



dataloader_train = DataLoader(train_data, batch_size=Batch_size)
dataloader_val = DataLoader(val_data, batch_size=Batch_size)

# Training

In [16]:

learning_rate= 0.01
startEpoch =0
numEpochs =15
model = CNNfdp(10).cuda()
criterion = nn.CrossEntropyLoss()
index =0
best_acc = 0
best_loss = 1
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
for epochs in range(startEpoch,numEpochs + 1):
  modelLoss_train = 0.0
  modelAcc_train = 0.0
  totalSize = 0
  for inputs,target in dataloader_train:
    optimizer.zero_grad()
    inputs = inputs.type(torch.FloatTensor).cuda()
    target = target.cuda()
    output = model(inputs)
    outp,preds = torch.max(output,1)
    loss = criterion(output,target)

    modelLoss_train += loss.item()*inputs.size(0)
    totalSize += inputs.size(0)
    modelAcc_train += torch.sum(preds == target).item()
    #print(preds)
    loss.backward()
    optimizer.step()
    index += 1

  modelLoss_epoch_train = modelLoss_train/totalSize
  modelAcc_epoch_train = modelAcc_train/totalSize

  #salvataggio stato
  torch.save(model.state_dict(), '/train_weights.pth')
  model.eval()

  totalSize_val = 0
  modelLoss_val = 0.0
  modelAcc_val = 0.0

  for inputs,labels in dataloader_val:
        inputs = inputs.type(torch.FloatTensor).cuda()
        labels = labels.cuda()

        y = model(inputs)
        outp, preds = torch.max(y,1)
        lossCNN = criterion(y,labels)

        modelLoss_val += lossCNN.item()*inputs.size(0)
        totalSize_val += inputs.size(0)
        modelAcc_val += torch.sum(preds == labels.data).item()

  modelLoss_epoch_val = modelLoss_val/totalSize_val
  modelAcc_epoch_val = modelAcc_val/totalSize_val
  print('Epoch[%d] Train Acc: %.4f , Loss: %.4f ; Validation Acc: %.4f , Loss: %.4f'%(epochs, modelAcc_epoch_train, modelLoss_epoch_train, modelAcc_epoch_val, modelLoss_epoch_val));

  if(modelAcc_epoch_val > best_acc) or (modelAcc_epoch_val == best_acc and modelLoss_epoch_val < best_loss):
        print('..... Saving best weights ....')
        best_acc = modelAcc_epoch_val
        best_loss = modelLoss_epoch_val
        best_epoca = epochs

        torch.save(model.state_dict(), '/content/best_model_weights.pth')

  with open('/content/lossVal.txt', "a") as file_object:
        file_object.write(str(modelLoss_epoch_val)+ '\n')

  with open('/content/AccVal.txt',"a") as file_object:
        file_object.write(str(modelAcc_epoch_val)+ '\n')

  with open('/content/lossTrain.txt', "a") as file_object:
        file_object.write(str(modelLoss_epoch_train)+ '\n')

  with open('/content/AccTrain.txt',"a") as file_object:
        file_object.write(str(modelAcc_epoch_train)+ '\n')

Epoch[0] Train Acc: 0.3107 , Loss: 1.8789 ; Validation Acc: 0.4491 , Loss: 1.5366
..... Saving best weights ....
Epoch[1] Train Acc: 0.4919 , Loss: 1.3897 ; Validation Acc: 0.5406 , Loss: 1.2908
..... Saving best weights ....
Epoch[2] Train Acc: 0.5830 , Loss: 1.1605 ; Validation Acc: 0.6205 , Loss: 1.0789
..... Saving best weights ....
Epoch[3] Train Acc: 0.6505 , Loss: 0.9847 ; Validation Acc: 0.6711 , Loss: 0.9417
..... Saving best weights ....
Epoch[4] Train Acc: 0.6994 , Loss: 0.8526 ; Validation Acc: 0.7023 , Loss: 0.8567
..... Saving best weights ....
Epoch[5] Train Acc: 0.7388 , Loss: 0.7416 ; Validation Acc: 0.7215 , Loss: 0.8059
..... Saving best weights ....
Epoch[6] Train Acc: 0.7759 , Loss: 0.6431 ; Validation Acc: 0.7318 , Loss: 0.7774
..... Saving best weights ....
Epoch[7] Train Acc: 0.8067 , Loss: 0.5535 ; Validation Acc: 0.7407 , Loss: 0.7633
..... Saving best weights ....
Epoch[8] Train Acc: 0.8378 , Loss: 0.4692 ; Validation Acc: 0.7440 , Loss: 0.7750
..... Saving b

# Testing

we will download the test set

In [17]:
test_data = torchvision.datasets.CIFAR10('/content/Cifar10_root', train=False)

now we apply to the test images the same operation that we applied to the train and validation sets

In [18]:

test_images=[]
convert_tensor = transforms.ToTensor()
for images,targets in test_data:
  test_images.append(convert_tensor(images))

In [19]:


norm = transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))
for elem in test_images:
  elem = norm(elem)

In [20]:
target_t=[]
for _,targets in test_data:
  target_t.append(targets)


In [21]:

test_set = MyDataset(test_images,target_t)
dataloader_test = DataLoader(test_set, batch_size=Batch_size)

load the best weights

In [22]:

model_best = CNNfdp(10)
model_best.load_state_dict(torch.load('/content/best_model_weights.pth'))
model_best.eval()

CNNfdp(
  (conv_layer1): Conv2d(3, 128, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
  (conv_layer2): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
  (conv_layer3): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
  (maxpool_lyr): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout_lyr): Dropout(p=0.25, inplace=False)
  (fc1): Linear(in_features=12800, out_features=128, bias=True)
  (fc_final): Linear(in_features=128, out_features=10, bias=True)
)

In [23]:
totalSize_test = 0
modelAcc_test = 0.0

for inputs,labels in dataloader_test:
        inputs = inputs.type(torch.FloatTensor).cuda()
        labels = labels.cuda()

        y = model(inputs)
        outp, preds = torch.max(y,1)
        lossCNN = criterion(y,labels)

        totalSize_test += inputs.size(0)
        modelAcc_test += torch.sum(preds == labels.data).item()

modelAcc_epoch_test = modelAcc_test/totalSize_test
print('Acc : %.4f'%modelAcc_epoch_test)

Acc : 0.7552
